In [78]:
# Import libraries
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from keras.layers import Input, Embedding, Reshape, Dot, Concatenate, Dense, Dropout

from scipy.sparse import coo_matrix

In [79]:
file_path = '../data/ratings_filtered.csv'
columns = ['user_id', 'movie_id', 'rating', 'timestamp']
df_ratings = pd.read_csv(file_path, names=columns).drop(0)
df_ratings.head(5)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_17552\211700579.py:3: DtypeWarning: Columns (0,1,2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_ratings = pd.read_csv(file_path, names=columns).drop(0)


,user_id,movie_id,rating,timestamp
1,1,110,1.0,1425941529
2,1,147,4.5,1425942435
3,1,858,5.0,1425941523
4,1,1246,5.0,1425941556
5,1,1968,4.0,1425942148


In [80]:
df_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11016557 entries, 1 to 11016557
Data columns (total 4 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   user_id    object
 1   movie_id   object
 2   rating     object
 3   timestamp  object
dtypes: object(4)
memory usage: 336.2+ MB


In [81]:
df_ratings.isnull().sum()

user_id      0
movie_id     0
rating       0
timestamp    0
dtype: int64

In [82]:
df_ratings.shape

(11016557, 4)

In [83]:
df_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11016557 entries, 1 to 11016557
Data columns (total 4 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   user_id    object
 1   movie_id   object
 2   rating     object
 3   timestamp  object
dtypes: object(4)
memory usage: 336.2+ MB


In [65]:
# Filter sparse movies
#min_movie_ratings = 10000
#filter_movies = (df_ratings['movie_id'].value_counts() > min_movie_ratings)
#filter_movies = filter_movies[filter_movies].index.tolist()
#
## Filter sparse users
#min_user_ratings = 200
#filter_users = (df_ratings['user_id'].value_counts() > min_user_ratings)
#filter_users = filter_users[filter_users].index.tolist()
#
## Actual filtering
#df_filterd = df_ratings[(df_ratings['movie_id'].isin(filter_movies)) & (df_ratings['user_id'].isin(filter_users))]
#del filter_movies, filter_users, min_movie_ratings, min_user_ratings
#print('Shape User-Ratings unfiltered:\t{}'.format(df_ratings.shape))
#print('Shape User-Ratings filtered:\t{}'.format(df_filterd.shape))

Shape User-Ratings unfiltered:	(11016557, 4)
Shape User-Ratings filtered:	(1460289, 4)


In [84]:
df_filterd = df_ratings[:7000000]
df_filterd = df_filterd.drop('timestamp', axis=1).sample(frac=1).reset_index(drop=True)

In [85]:
df_filterd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7000000 entries, 0 to 6999999
Data columns (total 3 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   user_id   object
 1   movie_id  object
 2   rating    object
dtypes: object(3)
memory usage: 160.2+ MB


# MODELAMIENTO

In [86]:
#df_train, df_test = train_test_split(df_filterd, test_size=0.2, random_state = 42)
# Testingsize
n = 100000

# Split train- & testset
df_train = df_filterd[:-n]
df_test = df_filterd[-n:]

In [ ]:
# Convert columns to integers
df_train['user_id'] = df_train['user_id'].astype(np.int64)
df_train['movie_id'] = df_train['movie_id'].astype(np.int64)
df_train['rating'] = df_train['rating'].astype(np.float64)

df_test['user_id'] = df_test['user_id'].astype(np.int64)
df_test['movie_id'] = df_test['movie_id'].astype(np.int64)
df_test['rating'] = df_test['rating'].astype(np.float64)

In [87]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6900000 entries, 0 to 6899999
Data columns (total 3 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   user_id   object
 1   movie_id  object
 2   rating    object
dtypes: object(3)
memory usage: 157.9+ MB


In [89]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 6900000 to 6999999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   user_id   100000 non-null  object
 1   movie_id  100000 non-null  object
 2   rating    100000 non-null  object
dtypes: object(3)
memory usage: 2.3+ MB


In [90]:
# Create user- & movie-id mapping
user_id_mapping = {id:i for i, id in enumerate(df_filterd['user_id'].unique())}
movie_id_mapping = {id:i for i, id in enumerate(df_filterd['movie_id'].unique())}

# Create correctly mapped train- & testset
train_user_data = df_train['user_id'].map(user_id_mapping)
train_movie_data = df_train['movie_id'].map(movie_id_mapping)

test_user_data = df_test['user_id'].map(user_id_mapping)
test_movie_data = df_test['movie_id'].map(movie_id_mapping)

# Get input variable-sizes
users = len(user_id_mapping)
movies = len(movie_id_mapping)

In [121]:
index_to_movieId = {i: id for i, id in enumerate(df_filterd['movie_id'].unique())}
with open('../data/movieId_to_index_embedding.json', 'w') as f:
    json.dump(movie_id_mapping, f)
with open('../data/index_to_movieId_embedding.json', 'w') as f:
    json.dump(index_to_movieId, f)

In [91]:
train_user_data

0               0
1               1
2               2
3               3
4               4
            ...  
6899995    146395
6899996     21373
6899997     16951
6899998     38068
6899999    102267
Name: user_id, Length: 6900000, dtype: int64

In [92]:
train_movie_data

0            0
1            1
2            2
3            3
4            4
          ... 
6899995    887
6899996    566
6899997    120
6899998    333
6899999    613
Name: movie_id, Length: 6900000, dtype: int64

In [95]:
#convert tran_user_data to int
#train_user_data.dropna(inplace=True)
#train_movie_data.dropna(inplace=True)
#test_user_data.dropna(inplace=True)
#test_movie_data.dropna(inplace=True)

#train_user_data = train_user_data.astype(np.int64)
#train_movie_data = train_movie_data.astype(np.int64)
#test_user_data = test_user_data.astype(np.int64)
#test_movie_data = test_movie_data.astype(np.int64)

df_train['user_id'] = df_train['user_id'].astype(np.int64)
df_train['movie_id'] = df_train['movie_id'].astype(np.int64)
df_train['rating'] = df_train['rating'].astype(np.float64)

df_test['user_id'] = df_test['user_id'].astype(np.int64)
df_test['movie_id'] = df_test['movie_id'].astype(np.int64)
df_test['rating'] = df_test['rating'].astype(np.float64)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_17552\4101516989.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['user_id'] = df_train['user_id'].astype(np.int64)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_17552\4101516989.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['movie_id'] = df_train['movie_id'].astype(np.int64)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_17552\4101516989.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

In [73]:
# view max avalue of testt
print(train_user_data.min(), train_user_data.max())
print(train_movie_data.min(), train_movie_data.max())

0 9429
0 309


In [74]:
train_user_data.info()

<class 'pandas.core.series.Series'>
RangeIndex: 1360289 entries, 0 to 1360288
Series name: user_id
Non-Null Count    Dtype
--------------    -----
1360289 non-null  int64
dtypes: int64(1)
memory usage: 10.4 MB


In [75]:
train_movie_data.info()

<class 'pandas.core.series.Series'>
RangeIndex: 1360289 entries, 0 to 1360288
Series name: movie_id
Non-Null Count    Dtype
--------------    -----
1360289 non-null  int64
dtypes: int64(1)
memory usage: 10.4 MB


In [96]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6900000 entries, 0 to 6899999
Data columns (total 3 columns):
 #   Column    Dtype  
---  ------    -----  
 0   user_id   int64  
 1   movie_id  int64  
 2   rating    float64
dtypes: float64(1), int64(2)
memory usage: 157.9 MB


In [97]:
# Setup variables
user_embedding_size = 20
movie_embedding_size = 10


##### Create model
# Set input layers
user_id_input = Input(shape=[1], name='user')
movie_id_input = Input(shape=[1], name='movie')

# Create embedding layers for users and movies
user_embedding = Embedding(output_dim=user_embedding_size, 
                           input_dim=users,
                           input_length=1, 
                           name='user_embedding')(user_id_input)
movie_embedding = Embedding(output_dim=movie_embedding_size, 
                            input_dim=movies,
                            input_length=1, 
                            name='item_embedding')(movie_id_input)

# Reshape the embedding layers
user_vector = Reshape([user_embedding_size])(user_embedding)
movie_vector = Reshape([movie_embedding_size])(movie_embedding)

# Concatenate the reshaped embedding layers
concat = Concatenate()([user_vector, movie_vector])

# Combine with dense layers
dense = Dense(256)(concat)
y = Dense(1)(dense)

# Setup model
model = Model(inputs=[user_id_input, movie_id_input], outputs=y)
model.compile(loss='mse', optimizer='adam')


# Fit model
history = model.fit([train_user_data, train_movie_data],
          df_train['rating'],
          batch_size=256, 
          epochs=1,
          validation_split=0.1,
          shuffle=True)

d:\OneDrive - Universidad Peruana de Ciencias\Documents\Carrera\2024-1\Applications of Data Science\CC219-TP-TF-2024-1-CC92\env\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


24258/24258 ━━━━━━━━━━━━━━━━━━━━ 655s 27ms/step - loss: 0.9724 - val_loss: 0.7886


In [98]:
# Test model
y_pred = model.predict([test_user_data, test_movie_data])
y_true = df_test['rating'].values

#  Compute RMSE
rmse = np.sqrt(mean_squared_error(y_pred=y_pred, y_true=y_true))
print('\n\nTesting Result With Keras Deep Learning: {:.4f} RMSE'.format(rmse))

3125/3125 ━━━━━━━━━━━━━━━━━━━━ 2s 686us/step


Testing Result With Keras Deep Learning: 0.8899 RMSE


In [99]:
model.summary()

Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ user (InputLayer)   │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ movie (InputLayer)  │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ user_embedding      │ (None, 1, 20)     │  3,372,680 │ user[0][0]        │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ item_embedding      │ (None, 1, 10)     │    104,170 │ movie[0][0]       │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_8 (Reshape) │ (None, 20)        │          0 │ user_embedding[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_9 (Reshape) │ (None, 10)        │          0 │ item_embedding[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_4       │ (None, 30)        │          0 │ reshape_8[0][0],  │
│ (Concatenate)       │                   │            │ reshape_9[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_8 (Dense)     │ (None, 256)       │      7,936 │ concatenate_4[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_9 (Dense)     │ (None, 1)         │        257 │ dense_8[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 10,455,131 (39.88 MB)

 Trainable params: 3,485,043 (13.29 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 6,970,088 (26.59 MB)

In [109]:
# view quantity of keys in movie_id_mapping
len(movie_id_mapping.keys())

10417

In [321]:
def predict_ratings_for_new_user(userId, new_user_interactions):
    movie_id_mapping = {}
    index_to_movieId = {}
    with open('../data/movieId_to_index_embedding.json', 'r') as file:
        movie_id_mapping = json.load(file)
    with open('../data/index_to_movieId_embedding.json', 'r') as file:
        index_to_movieId = json.load(file)
    movie_id_mapping = {int(movie_id): index for movie_id, index in movie_id_mapping.items()}
    index_to_movieId = {int(index): movie_id for index, movie_id in index_to_movieId.items()}
    
    # Ids de todas las peliculas
    movie_ids = np.arange(10417)

    # Convert the movie IDs to the internal IDs used by the model
    movies_already_seen = []
    for movie_id, rating in new_user_interactions.items():
        index = movie_id_mapping[int(movie_id)]
        movies_already_seen.append(index)
    
    # Filtrar las películas que el usuario ya ha visto
    movie_ids_to_predict = np.setdiff1d(movie_ids, movies_already_seen)
    #print(movie_ids_to_predict)
    user_id_array = np.array([userId] * (10417 - len(new_user_interactions)))
    #print(user_id_array)
    # Generar predicciones para las películas que el usuario no ha visto
    predicted_ratings = model.predict([user_id_array, movie_ids_to_predict])
    # Ordenar las predicciones de mayor a menor
    top_indices = np.argsort(predicted_ratings[:, 0])[::-1]

    # Seleccionar los índices de las 10 mejores predicciones
    top_10_indices = top_indices[:10]
    #print(top_10_indices)
    ## Imprimir las 10 mejores predicciones
    #print(predicted_ratings[top_10_indices, 0])
    # Ordenar las películas por calificación predicha de mayor a menor
    #recommended_movie_indices = np.argsort(predicted_ratings[:, 0])[::-1]
    ## Seleccionar los índices de las 10 películas más altas
    #top_recommended_indices = recommended_movie_indices[:10]  # Selecciona solo los 10 primeros
    
    # Convertir los índices de las películas recomendadas a sus IDs originales
    recommended_movie_ids = [index_to_movieId[int(index)] for index in top_10_indices]
    return recommended_movie_ids
    

In [322]:
# Films that the user has seen
interactions = {'6520': 3, '27793': 3, '100': 3, '200': 3, '123': 3, '1248': 3}
userId = 100
print(f"IDs de películas recomendadas: {predict_ratings_for_new_user(userId, interactions)}")

[    0     1     2 ... 10414 10415 10416]
[100 100 100 ... 100 100 100]
326/326 ━━━━━━━━━━━━━━━━━━━━ 0s 703us/step
[  65 4594 5248  899 2562 6234  599 1302  253 1701]
[4.599387  4.5956903 4.5909376 4.543286  4.537885  4.536967  4.5239954
 4.5164485 4.511561  4.5093236]
IDs de películas recomendadas: [318, '1942', '26947', '318', 27033, 93834, 922, 926, 858, '1245']


In [323]:
# Films that the user has seen
interactions = {'123': 3}
userId = 1
print(f"IDs de películas recomendadas: {predict_ratings_for_new_user(userId, interactions)}")

[    0     1     2 ... 10414 10415 10416]
[1 1 1 ... 1 1 1]
326/326 ━━━━━━━━━━━━━━━━━━━━ 0s 904us/step
[  65 4596 5251  899 2562 6238  599 1302  253 3464]
[4.454901  4.451204  4.4464517 4.3988004 4.3933997 4.3924804 4.3795094
 4.371962  4.3670745 4.366213 ]
IDs de películas recomendadas: [318, 4806, 96966, '318', 27033, 167112, 922, 926, 858, '1248']


In [229]:
interactions = {'123': 3, '1248': 3}
user_id_for_recommendation = 1
# Ids de todas las peliculas
movie_ids = np.arange(309)
new_user_interactions = {int(movie_id): rating for movie_id, rating in interactions.items()}
movies_already_seen = []

for movie_id, rating in new_user_interactions.items():
    index = movie_id_mapping[movie_id]
    movies_already_seen.append(index)
    
# Filtrar las películas que el usuario ya ha visto
movie_ids_to_predict = np.setdiff1d(movie_ids, movies_already_seen)
# Generar predicciones para las películas que el usuario no ha visto
user_id_array = np.array([user_id_for_recommendation] * len(movie_ids_to_predict))
predicted_ratings = model.predict([user_id_array, movie_ids_to_predict])

# Ordenar las películas por calificación predicha de mayor a menor
recommended_movie_indices = np.argsort(predicted_ratings[:, 0])[::-1]
# Seleccionar los índices de las 10 películas más altas
top_recommended_indices = recommended_movie_indices[:10]  # Selecciona solo los 10 primeros

# Convertir los índices de las películas recomendadas a sus IDs originales si es necesario
top_recommended_movie_ids = movie_ids_to_predict[top_recommended_indices]  # Ajusta esto según tu mapeo de IDs

print(f"IDs de películas recomendadas para el usuario {user_id_for_recommendation}: {top_recommended_movie_ids}")

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 889us/step
IDs de películas recomendadas para el usuario 1: [ 65 253  51   2 215  90  14 290 196  38]


In [214]:
interactions = {'6520': 3, '27793': 3}
user_id_for_recommendation = 1
# Ids de todas las peliculas
movie_ids = np.arange(309)
new_user_interactions = {int(movie_id): rating for movie_id, rating in interactions.items()}
movies_already_seen = []

for movie_id, rating in new_user_interactions.items():
    index = movie_id_mapping[movie_id]
    movies_already_seen.append(index)
    
# Filtrar las películas que el usuario ya ha visto
movie_ids_to_predict = np.setdiff1d(movie_ids, movies_already_seen)
# Generar predicciones para las películas que el usuario no ha visto
user_id_array = np.array([user_id_for_recommendation] * len(movie_ids_to_predict))
predicted_ratings = model.predict([user_id_array, movie_ids_to_predict])

# Ordenar las películas por calificación predicha de mayor a menor
recommended_movie_indices = np.argsort(predicted_ratings[:, 0])[::-1]
# Seleccionar los índices de las 10 películas más altas
top_recommended_indices = recommended_movie_indices[:10]  # Selecciona solo los 10 primeros

# Convertir los índices de las películas recomendadas a sus IDs originales si es necesario
top_recommended_movie_ids = movie_ids_to_predict[top_recommended_indices]  # Ajusta esto según tu mapeo de IDs

print(f"IDs de películas recomendadas para el usuario {user_id_for_recommendation}: {top_recommended_movie_ids}")

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 892us/step
IDs de películas recomendadas para el usuario 1: [ 65 253  51   2 215  90  14 290 196  38]


In [235]:
# Id de usuario
user_id_for_recommendation = 4  # Asegúrate de que este sea el índice mapeado, no el ID original
# TODO AJUSTAR EL ID

movie_ids = np.arange(309)  # Ajusta esto para incluir un rango más amplio de películas
# Lista de películas que el usuario ya ha visto (estos deberían ser índices mapeados, no IDs originales)
movies_already_seen = [0, 50, 28, 3] 

# Filtrar las películas que el usuario ya ha visto
movie_ids_to_predict = np.setdiff1d(movie_ids, movies_already_seen)
# Generar predicciones para las películas que el usuario no ha visto
user_id_array = np.array([user_id_for_recommendation] * len(movie_ids_to_predict))
predicted_ratings = model.predict([user_id_array, movie_ids_to_predict])

# Ordenar las películas por calificación predicha de mayor a menor
recommended_movie_indices = np.argsort(predicted_ratings[:, 0])[::-1]

# Seleccionar los índices de las 10 películas más altas
top_recommended_indices = recommended_movie_indices[:10]  # Selecciona solo los 10 primeros

# Convertir los índices de las películas recomendadas a sus IDs originales si es necesario
top_recommended_movie_ids = movie_ids_to_predict[top_recommended_indices]  # Ajusta esto según tu mapeo de IDs

print(f"IDs de películas recomendadas para el usuario {user_id_for_recommendation}: {top_recommended_movie_ids}")

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 778us/step
IDs de películas recomendadas para el usuario 4: [ 65 253  51   2 215  90  14 290 196  38]


In [141]:
# Save the model
model.save('../embeddings.h5')